In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip').fillna(' ')

train.head() # (159571, 8) total dataset.

In [ ]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip').fillna(' ').sample(1000)
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip').fillna(' ').sample(1000)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

train = train.loc[:,class_names]


In [ ]:
print("TFIDF")
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    norm='l2',
    min_df=0,
    smooth_idf=False,
    max_features=15000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

# char_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='char',
#     stop_words='english',
#     ngram_range=(2, 6),
#     norm='l2',
#     min_df=0,
#     smooth_idf=False,
#     max_features=50000)
# char_vectorizer.fit(all_text)
# train_char_features = char_vectorizer.transform(train_text)
# test_char_features = char_vectorizer.transform(test_text)

# train_features = hstack([train_char_features, train_word_features])
# del train_char_features,train_word_features
# test_features = hstack([test_char_features, test_word_features])
# del test_char_features,test_word_features

train_features = train_word_features
test_features = test_word_features

print(train_features.shape)
print(test_features.shape)
d_test = xgb.DMatrix(test_features)
del test_features
gc.collect()
print(" End vectorizor")



In [ ]:
# print("Modeling starts..")
# cv_scores = []
# xgb_preds = []
# # submission = pd.DataFrame.from_dict({'id': test['id']})
# for class_name in class_names:
#     train_target = train[class_name]
#     # Split out a validation set
#     X_train, X_valid, y_train, y_valid = train_test_split(
#         train_features, train_target, test_size=0.25, random_state=23)

#     xgb_params = {'eta': 0.1, 
#               'max_depth': 5, 
#               'subsample': 0.8, 
#               'colsample_bytree': 0.8,
#               'tree_method': 'gpu_hist',
#               "predictor": "gpu_predictor",
#               'booster': 'gbtree',
#               'n_estimators':100,
#               'objective': 'multi:softprob', 
#               'num_class': 6
#              } ## Main problem with multiclass is eval_matrics. better not to define manually.
    

#     d_train = xgb.DMatrix(X_train, y_train)
#     d_valid = xgb.DMatrix(X_valid, y_valid)

#     watchlist = [(d_valid, 'valid')]
#     model = xgb.train(xgb_params, d_train, 100, watchlist, verbose_eval=False, early_stopping_rounds=30)
#     print("class Name: {}".format(class_name))
# #     print(model.attributes()['best_msg'])
#     cv_scores.append(float(model.attributes()['best_score']))
# #     submission[class_name] = model.predict(d_test)
#     del X_train, X_valid, y_train, y_valid
#     gc.collect()
# print('Total CV score is {}'.format(np.mean(cv_scores)))
# # submission.to_csv('submission.csv', index=False)

In [ ]:
clf = XGBClassifier(learning_rate =0.05,
                    n_estimators=300,
                    max_depth=10,
                    min_child_weight=1,
                    gamma=0.5,
                    reg_alpha=0,              
                    reg_lambda=2,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    scale_pos_weight=1,
                    objective='multi:softmax',
                    num_class=14,
                    nthread=20,
                    seed=1000)
xgb_param = clf.get_xgb_params()
xgTrain = xgb.DMatrix(x_train_, label=y_train_)

print ('Start cross validation')
cvresult = xgb.cv(xgb_param, xgTrain, num_boost_round=500, nfold=5, metrics=['mlogloss'],
     early_stopping_rounds=5, stratified=True, seed=1000)


print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])
print('Fit on the all_trainingsdata')
clf.fit(X_train,y_train,eval_metric=['mlogloss'])

In [ ]:
for class_name in class_names:
    train_target = train[class_name]
    # Split out a validation set
    X_train, X_valid, y_train, y_valid = train_test_split(
        train_features, train_target, test_size=0.25, random_state=23)
    
    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)
    
    clf = XGBClassifier(learning_rate =0.05,
                    n_estimators=300,
                    max_depth=10,
                    min_child_weight=1,
                    gamma=0.5,
                    reg_alpha=0,              
                    reg_lambda=2,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    tree_method = 'gpu_hist',
                    predictor= "gpu_predictor",
                    objective='multi:softmax',
                    num_class=14,
                    nthread=20,
                    seed=1000)
xgb_param = clf.get_xgb_params()
# xgTrain = xgb.DMatrix(x_train_, label=y_train_)

print ('Start cross validation')
cvresult = xgb.cv(xgb_param, d_train, num_boost_round=500, nfold=5, metrics=['mlogloss'],
     early_stopping_rounds=5, stratified=True, seed=1000)


print('Best number of trees = {}'.format(cvresult.shape[0]))
clf.set_params(n_estimators=cvresult.shape[0])
print('Fit on the all_trainingsdata')
clf.fit(X_train,y_train,eval_metric=['mlogloss'])

y_pred = model.predict(d_valid) 

predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_valid, predictions) 

print("Accuracy: %.2f%%" % (accuracy * 100.0))


In [ ]:
print(y_pred)

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))